In [2]:
import cv2 as cv
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [3]:
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

In [4]:
main_dir = 'C:\\Users\\nguye\\PycharmProjects\\DPL302\\Pose recognition'

In [5]:
label = 'Yoga'
labels_path = os.path.join(main_dir,'dataset',label)
labels = os.listdir(labels_path)

In [6]:
poses = np.array(labels)
print(poses)

['adho mukha svanasana' 'adho mukha vriksasana' 'agnistambhasana'
 'ananda balasana' 'anantasana' 'anjaneyasana' 'ardha bhekasana'
 'ardha chandrasana' 'ardha matsyendrasana' 'ardha pincha mayurasana'
 'ardha uttanasana' 'ashtanga namaskara' 'astavakrasana' 'baddha konasana'
 'bakasana' 'balasana' 'bhairavasana' 'bharadvajasana i' 'bhekasana'
 'bhujangasana' 'bhujapidasana' 'bitilasana' 'camatkarasana'
 'chakravakasana' 'chaturanga dandasana' 'dandasana' 'dhanurasana'
 'durvasasana' 'dwi pada viparita dandasana' 'eka pada koundinyanasana i'
 'eka pada koundinyanasana ii' 'eka pada rajakapotasana'
 'eka pada rajakapotasana ii' 'ganda bherundasana' 'garbha pindasana'
 'garudasana' 'gomukhasana' 'halasana' 'hanumanasana' 'janu sirsasana'
 'kapotasana' 'krounchasana' 'kurmasana' 'lolasana'
 'makara adho mukha svanasana' 'makarasana' 'malasana' 'marichyasana i'
 'marichyasana iii' 'marjaryasana' 'matsyasana' 'mayurasana'
 'natarajasana' 'padangusthasana' 'padmasana' 'parighasana'
 'paripurn

In [27]:
cap = cv.VideoCapture(0)
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()

        frame.flags.writeable = False
        frame = cv.cvtColor(frame, cv.COLOR_BGR2RGB) 
        results = pose.process(frame)
        frame.flags.writeable = True
        frame = cv.cvtColor(frame, cv.COLOR_RGB2BGR)
        
        mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        cv.imshow('OpenCV Feed', cv.flip(frame,1))

        if cv.waitKey(10) & 0xFF == ord('q'):
            break
            
    cap.release()
    cv2.destroyAllWindows()

In [141]:
for pose_action in poses:
    try: 
        os.makedirs(os.path.join(main_dir, 'keypoints',pose_action))
    except:
        pass

In [142]:
img_no = 0
with mp_pose.Pose(static_image_mode = True, min_detection_confidence=0.5) as pose:
    for pose_actions in poses:
        current_pose = pose_actions
        pose_path = os.path.join(main_dir,'dataset', label, pose_actions)
        pose_actions = np.array(os.listdir(pose_path))
        img_no = 0
        for img in pose_actions:
            img_path = os.path.join(main_dir,'dataset', label, current_pose, img)
            frame = cv.imread(img_path)
            
            try:
                frame.flags.writeable = False
                frame = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
                results = pose.process(frame)
                frame.flags.writeable = True
                frame = cv.cvtColor(frame, cv.COLOR_RGB2BGR)
            except:
                print('Error while extracting ', img, 'of', current_pose)
                img_no += 1
                pass
            
            keypoints = np.array([[result.x, result.y, result.z, result.visibility] for result in results.pose_landmarks.landmark]).flatten()
            
            kp_path = os.path.join(main_dir, 'keypoints', current_pose, str(img_no))
            np.save(kp_path, keypoints)
            img_no += 1
                
            cv.imshow('Reading', frame)
            cv.waitKey(100)


cv.destroyAllWindows()

In [7]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [8]:
label_map = {label:num for num, label in enumerate(poses)}

In [9]:
label_map

{'adho mukha svanasana': 0,
 'adho mukha vriksasana': 1,
 'agnistambhasana': 2,
 'ananda balasana': 3,
 'anantasana': 4,
 'anjaneyasana': 5,
 'ardha bhekasana': 6,
 'ardha chandrasana': 7,
 'ardha matsyendrasana': 8,
 'ardha pincha mayurasana': 9,
 'ardha uttanasana': 10,
 'ashtanga namaskara': 11,
 'astavakrasana': 12,
 'baddha konasana': 13,
 'bakasana': 14,
 'balasana': 15,
 'bhairavasana': 16,
 'bharadvajasana i': 17,
 'bhekasana': 18,
 'bhujangasana': 19,
 'bhujapidasana': 20,
 'bitilasana': 21,
 'camatkarasana': 22,
 'chakravakasana': 23,
 'chaturanga dandasana': 24,
 'dandasana': 25,
 'dhanurasana': 26,
 'durvasasana': 27,
 'dwi pada viparita dandasana': 28,
 'eka pada koundinyanasana i': 29,
 'eka pada koundinyanasana ii': 30,
 'eka pada rajakapotasana': 31,
 'eka pada rajakapotasana ii': 32,
 'ganda bherundasana': 33,
 'garbha pindasana': 34,
 'garudasana': 35,
 'gomukhasana': 36,
 'halasana': 37,
 'hanumanasana': 38,
 'janu sirsasana': 39,
 'kapotasana': 40,
 'krounchasana': 

In [34]:
poses_kp, labels_kp = [], []
for pose_action in poses:
    pose_path = os.path.join(main_dir, 'keypoints', pose_action)
    pose_count = len(os.listdir(pose_path))
    for pose_kp in range(pose_count):
        keypoints = np.load(os.path.join(main_dir, 'keypoints', pose_action,'{}.npy'.format(pose_kp)))
        poses_kp.append(keypoints)
        labels_kp.append(label_map[pose_action])
    

In [35]:
np.array(poses_kp).shape

(5991, 132)

In [36]:
np.array(labels_kp).shape

(5991,)

In [37]:
X = np.array(poses_kp)
X.shape

(5991, 132)

In [38]:
y = to_categorical(labels_kp).astype(int)
y.shape

(5991, 107)

In [39]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.05)

In [45]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

In [46]:
model = Sequential()
model.add(Conv1D(filters = 64, kernel_size = 3, activation='relu',input_shape=(132,1)))
model.add(Conv1D(filters = 64, kernel_size = 3, activation='relu'))
model.add(MaxPooling1D(pool_size = 2))
model.add(Flatten())
model.add(Dense(128, activation= 'relu'))
model.add(Dense(107, activation= 'softmax'))

In [47]:
model.compile(loss='categorical_crossentropy', optimizer ='adam', metrics = ['accuracy'])

In [48]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_4 (Conv1D)           (None, 130, 64)           256       
                                                                 
 conv1d_5 (Conv1D)           (None, 128, 64)           12352     
                                                                 
 max_pooling1d_2 (MaxPooling  (None, 64, 64)           0         
 1D)                                                             
                                                                 
 flatten_2 (Flatten)         (None, 4096)              0         
                                                                 
 dense_4 (Dense)             (None, 128)               524416    
                                                                 
 dense_5 (Dense)             (None, 107)               13803     
                                                      

In [49]:
model.fit(X_train, y_train, epochs = 35, verbose = True)

Epoch 1/50
178/178 [==============================] - 6s 27ms/step - loss: 4.0104 - accuracy: 0.1093
Epoch 2/50
178/178 [==============================] - 5s 28ms/step - loss: 2.3068 - accuracy: 0.4112
Epoch 3/50
178/178 [==============================] - 5s 27ms/step - loss: 1.7197 - accuracy: 0.5435
Epoch 4/50
178/178 [==============================] - 5s 29ms/step - loss: 1.4573 - accuracy: 0.6055
Epoch 5/50
178/178 [==============================] - 5s 29ms/step - loss: 1.2941 - accuracy: 0.6575
Epoch 6/50
178/178 [==============================] - 5s 28ms/step - loss: 1.1915 - accuracy: 0.6763
Epoch 7/50
178/178 [==============================] - 5s 29ms/step - loss: 1.0969 - accuracy: 0.6944
Epoch 8/50
178/178 [==============================] - 5s 30ms/step - loss: 1.0276 - accuracy: 0.7164
Epoch 9/50
178/178 [==============================] - 5s 29ms/step - loss: 0.9752 - accuracy: 0.7254
Epoch 10/50
178/178 [==============================] - 5s 29ms/step - loss: 0.9211 - accura

In [54]:
model.save('model.h5')

In [55]:
model.load_weights('model.h5')

In [50]:
pred = model.predict(X_test)
y_pred = pred.argmax(axis = -1).tolist()
print(y_pred)

10/10 [==============================] - 0s 7ms/step
[65, 104, 60, 63, 79, 5, 59, 10, 106, 98, 33, 77, 24, 36, 78, 8, 84, 8, 40, 22, 31, 103, 98, 54, 62, 67, 73, 34, 2, 62, 85, 60, 100, 63, 10, 8, 15, 26, 32, 64, 105, 50, 7, 61, 77, 46, 100, 77, 19, 101, 45, 43, 85, 79, 1, 7, 54, 105, 69, 8, 32, 41, 95, 78, 45, 12, 59, 85, 81, 41, 73, 91, 51, 60, 29, 8, 68, 23, 87, 68, 85, 70, 47, 24, 35, 85, 51, 51, 33, 68, 54, 54, 3, 54, 103, 23, 85, 7, 93, 105, 67, 41, 3, 41, 59, 97, 81, 54, 66, 16, 25, 1, 50, 85, 3, 31, 91, 93, 14, 47, 1, 10, 84, 30, 103, 73, 44, 82, 66, 56, 85, 8, 26, 24, 21, 6, 88, 60, 66, 82, 27, 89, 35, 48, 99, 35, 80, 2, 100, 66, 91, 65, 102, 85, 5, 26, 33, 98, 13, 21, 21, 20, 6, 0, 57, 55, 29, 44, 63, 81, 16, 29, 28, 72, 80, 22, 85, 12, 39, 36, 27, 74, 85, 3, 100, 83, 33, 70, 25, 92, 69, 36, 79, 54, 44, 56, 23, 72, 87, 2, 72, 71, 97, 50, 40, 87, 76, 63, 32, 68, 95, 31, 93, 97, 12, 73, 85, 103, 42, 68, 87, 21, 74, 69, 36, 78, 29, 21, 96, 26, 26, 59, 100, 31, 10, 85, 5, 68, 61,

In [51]:
y_test = y_test.argmax(axis = -1).tolist()
print(y_test)

[105, 104, 60, 63, 79, 5, 59, 10, 37, 98, 33, 77, 24, 36, 78, 8, 84, 48, 40, 22, 31, 103, 98, 2, 62, 67, 73, 34, 17, 62, 63, 20, 52, 63, 12, 8, 15, 26, 32, 64, 105, 50, 7, 61, 72, 51, 93, 91, 19, 101, 45, 43, 54, 79, 71, 7, 54, 105, 69, 8, 77, 41, 95, 78, 45, 12, 59, 27, 47, 41, 70, 91, 51, 60, 30, 17, 19, 21, 19, 12, 85, 70, 47, 24, 35, 16, 51, 51, 33, 68, 54, 54, 99, 54, 97, 23, 37, 7, 93, 105, 73, 41, 15, 41, 96, 97, 81, 75, 88, 16, 56, 1, 50, 7, 3, 32, 91, 8, 14, 47, 1, 20, 84, 29, 97, 73, 64, 54, 66, 56, 16, 47, 26, 24, 74, 6, 88, 60, 66, 13, 27, 89, 35, 48, 99, 35, 80, 2, 100, 66, 91, 65, 102, 85, 5, 26, 71, 98, 13, 23, 49, 20, 41, 0, 57, 55, 30, 67, 39, 81, 16, 30, 28, 12, 80, 22, 85, 12, 39, 36, 27, 74, 43, 15, 100, 20, 33, 70, 103, 29, 87, 36, 79, 74, 44, 56, 23, 72, 87, 75, 72, 71, 17, 50, 89, 87, 76, 63, 32, 68, 96, 32, 93, 97, 12, 73, 60, 97, 42, 24, 87, 21, 74, 69, 36, 78, 88, 21, 59, 26, 19, 59, 100, 31, 10, 91, 5, 68, 61, 45, 89, 42, 102, 35, 78, 26, 66, 2, 88, 24, 98, 1

In [52]:
from sklearn.metrics import accuracy_score

In [53]:
accuracy_score(y_test, y_pred)

0.6933333333333334

In [24]:
image = 'C:\\Users\\nguye\\PycharmProjects\\DPL302\\Pose recognition\\dataset\\Yoga\\dandasana\\28-0.png'
with mp_pose.Pose(static_image_mode = True, min_detection_confidence=0.5) as pose:
    while True:
        frame = cv.imread(image)

        frame.flags.writeable = False
        frame = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
        results = pose.process(frame)
        frame.flags.writeable = True
        frame = cv.cvtColor(frame, cv.COLOR_RGB2BGR)

        keypoints = np.array([[result.x, result.y, result.z, result.visibility] for result in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
        
        mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        
        pred = model.predict(keypoints)
        y_pred = pred.argmax(axis=-1)
        
        print(y_pred)
        
        cv.imshow('kp',frame)
        
        if cv.waitKey(5) & 0xFF == ord('q'):
            break

cv.destroyAllWindows()



InvalidArgumentError: Graph execution error:

Detected at node 'sequential/conv1d/Conv1D/ExpandDims' defined at (most recent call last):
    File "C:\Users\nguye\AppData\Local\Programs\Python\Python39\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\nguye\AppData\Local\Programs\Python\Python39\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "c:\users\nguye\pycharmprojects\dpl302\venv\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "c:\users\nguye\pycharmprojects\dpl302\venv\lib\site-packages\traitlets\config\application.py", line 1043, in launch_instance
      app.start()
    File "c:\users\nguye\pycharmprojects\dpl302\venv\lib\site-packages\ipykernel\kernelapp.py", line 725, in start
      self.io_loop.start()
    File "c:\users\nguye\pycharmprojects\dpl302\venv\lib\site-packages\tornado\platform\asyncio.py", line 195, in start
      se

In [ ]:
img_no = 0
with mp_pose.Pose(static_image_mode = True, min_detection_confidence=0.5) as pose:
    for pose_actions in poses:
        current_pose = pose_actions
        pose_path = os.path.join(main_dir,'dataset', label, pose_actions)
        pose_actions = np.array(os.listdir(pose_path))
        img_no = 0
        for img in pose_actions:
            img_path = os.path.join(main_dir,'dataset', label, current_pose, img)
            frame = cv.imread(img_path)
            
            try:
                frame.flags.writeable = False
                frame = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
                results = pose.process(frame)
                frame.flags.writeable = True
                frame = cv.cvtColor(frame, cv.COLOR_RGB2BGR)
            except:
                print('Error while extracting ', img, 'of', current_pose)
                img_no += 1
                pass
            
            keypoints = np.array([[result.x, result.y, result.z, result.visibility] for result in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
            
            kp_path = os.path.join(main_dir, 'keypoints', current_pose, str(img_no))
            np.save(kp_path, keypoints)
            img_no += 1
                
            cv.imshow('Reading', frame)
            cv.waitKey(100)


cv.destroyAllWindows()